# Routing
##routing in langgraph have a ability to determine which node to execute next based on the current state and output of the node.

In [6]:
import os
from dotenv import load_dotenv
from langchain_groq import ChatGroq
from langgraph.graph import START,StateGraph,END

load_dotenv()
os.environ["GROQ_API_kEY"]= os.getenv("GROQ_API_kEY")

llm=ChatGroq(model="llama-3.1-8b-instant")

In [10]:
from typing_extensions import Literal,TypedDict
from pydantic import BaseModel,Field
from langchain_core.messages import HumanMessage,SystemMessage


#Structured Output class using Pydantic
class Router(BaseModel):
    step:Literal["poem","story","joke"]=Field(description="The next step in the routing process")
    

# Binding the router to the LLM 
llm_router=llm.with_structured_output(Router)


#State Class
class State(TypedDict):
    input:str
    decision:str
    output:str
    
    
def llm_call_1(state:State):
    """Write a story"""
    
    result=llm.invoke(state["input"])
    return{"output":result.content} 

def llm_call_2(state:State):
    """Write a joke"""
    print("Llm call 2 is called")
    result=llm.invoke(state["input"])
    return{"output":result.content} 

def llm_call_3(state:State):
    """Write a poem"""
    result=llm.invoke(state["input"])
    return{"output":result.content} 



def llm_call_router(state:State):
    """Route the input to the appropriate node"""
    
    decision=llm_router.invoke(
        [
             SystemMessage(
                 content="Route the input to story,joke or poem based on the users request"
             ),
             HumanMessage(content=state["input"])
        ]
           
    )
    print("LLM routing decision:", decision.step)
    return{"decision":decision.step}

def route_decision(state:State):
    if state["decision"] == "story":
        return "llm_call_1"
    
    if state["decision"] == "joke":
        return "llm_call_2"
    
    if state["decision"] == "poem":
        return "llm_call_3"
    
    
#Build Workflow
router=StateGraph(State)

#Add nodes    
router.add_node("llm_call_1",llm_call_1)
router.add_node("llm_call_2",llm_call_2)
router.add_node("llm_call_3",llm_call_3)
router.add_node("llm_call_router",llm_call_router)

router.add_edge(START, "llm_call_router")

# Router -> Conditional Choice
router.add_conditional_edges(
    "llm_call_router",
    route_decision,
    {
        "llm_call_1": "llm_call_1",
        "llm_call_2": "llm_call_2",
        "llm_call_3": "llm_call_3",
    }
)

# End the flow after the LLM calls
router.add_edge("llm_call_1", END)
router.add_edge("llm_call_2", END)
router.add_edge("llm_call_3", END)

# Compile the graph
app = router.compile()



In [12]:
result=app.invoke({"input": "Tell me something serious inshort"})
print(result["output"])

The world's biodiversity is experiencing a massive decline, with approximately 1 million species facing extinction due to human activities, primarily habitat destruction, climate change, and pollution.
